In [ ]:
# Parameters from Papermill
ORIGIN_ADDRESS = None
DESTINATION_ADDRESS = None
GOOGLE_API_KEY = None
OUTPUT_TOPOLOGY_PATH = None

MINIO_ENDPOINT = None
MINIO_ACCESS_KEY = None
MINIO_SECRET_KEY = None


In [ ]:
import googlemaps
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime
import s3fs
import json


In [ ]:
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)


In [ ]:
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1_rad, lat2_rad = np.radians(lat1), np.radians(lat2)
    dlon_rad = np.radians(lon2 - lon1)
    x = np.sin(dlon_rad) * np.cos(lat2_rad)
    y = np.cos(lat1_rad) * np.sin(lat2_rad) - \
        np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(dlon_rad)
    bearing = np.degrees(np.arctan2(x, y))
    return (bearing + 360) % 360


In [ ]:
def build_route_from_google(origin, destination):

    # Fetch directions
    directions = gmaps.directions(
        origin,
        destination,
        mode="driving",
        departure_time=datetime.utcnow()
    )

    if not directions:
        raise ValueError("No route returned from Google Directions API.")

    leg = directions[0]["legs"][0]
    duration_sec = leg["duration"]["value"]

    # Decode polyline into lat/lon points
    polyline = directions[0]["overview_polyline"]["points"]
    points = googlemaps.convert.decode_polyline(polyline)

    df_route = pd.DataFrame(points)
    df_route.rename(columns={"lat": "position_lat", "lng": "position_long"}, inplace=True)

    # Add timestamps (approx evenly spaced)
    n = len(df_route)
    df_route["timestamp"] = pd.to_datetime(datetime.utcnow()) + pd.to_timedelta(
        np.linspace(0, duration_sec, n), unit="s"
    )

    # Elevation API
    elevations = []
    batch_size = 256
    for i in range(0, n, batch_size):
        batch = df_route.iloc[i:i+batch_size]
        locations = list(zip(batch["position_lat"], batch["position_long"]))
        elev_res = gmaps.elevation(locations)
        elevations.extend([e["elevation"] for e in elev_res])
    df_route["enhanced_altitude"] = elevations

    return df_route


In [ ]:
def engineer_static_features_for_route(df):

    # Distance per segment
    distances = [0.0]
    for i in range(1, len(df)):
        coord1 = (df.loc[i-1, "position_lat"], df.loc[i-1, "position_long"])
        coord2 = (df.loc[i, "position_lat"], df.loc[i, "position_long"])
        distances.append(geodesic(coord1, coord2).meters)
    df["distance_m"] = distances
    df["distance_cum_m"] = df["distance_m"].cumsum()

    # Elevation gain
    df["elev_gain_m"] = df["enhanced_altitude"].diff().fillna(0)

    # Bearing
    bearings = [0.0]
    for i in range(1, len(df)):
        b = calculate_bearing(
            df.loc[i-1, "position_lat"], df.loc[i-1, "position_long"],
            df.loc[i, "position_lat"], df.loc[i, "position_long"],
        )
        bearings.append(b)
    df["bearing"] = bearings

    # Heading change + turns
    df["heading_change"] = df["bearing"].diff().abs().fillna(0)
    df["heading_change"] = df["heading_change"].apply(lambda x: min(x, 360 - x))

    df["is_turn"] = (df["heading_change"] > 30).astype(int)
    df["turn_count"] = df["is_turn"].rolling(30, min_periods=1).sum()

    return df


In [ ]:
df_route = build_route_from_google(ORIGIN_ADDRESS, DESTINATION_ADDRESS)
df_route = engineer_static_features_for_route(df_route)

df_route.head()


In [ ]:
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

with fs.open(OUTPUT_TOPOLOGY_PATH, 'w') as f:
    df_route.to_csv(f, index=False)

print("Saved topology to:", OUTPUT_TOPOLOGY_PATH)
